# compute the mean and std for dataset

In [1]:
import argparse
import yaml
import logging

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import wandb
import copy

from datasets.dataloaders import get_dataset
from datasets.transforms import data_transform
from models.models import LinearModel_MNIST, LinearModel_CIFAR10, LinearModel_CIFAR100
from utils.loss import C2Loss_Classification, gradient_centralization

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
info = logger.info

# Argument Parser
parser = argparse.ArgumentParser(description='Autoencoder Training')
parser.add_argument('--yaml', type=str, default='./yaml/linear_cifar100_ccl.yaml', help='Path to the YAML configuration file')
args, _ = parser.parse_known_args()

with open(args.yaml, 'r') as file:
    yaml_config = yaml.safe_load(file)

# Set default arguments from YAML
parser.set_defaults(**yaml_config['method'])
parser.set_defaults(**yaml_config['dataset'])
parser.set_defaults(**yaml_config['training'])
args, _ = parser.parse_known_args()

train_loader, val_loader, test_loader = get_dataset(args)

/home/ck696/.conda/envs/H3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/ck696/.conda/envs/H3/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")


Files already downloaded and verified
Files already downloaded and verified


In [2]:
mean = 0.
std = 0.
for images, _ in train_loader:
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)



NameError: name 'loader' is not defined

In [3]:
mean /= len(train_loader.dataset)
std /= len(train_loader.dataset)

In [21]:
import torch
from torchvision import datasets, transforms

print("MNIST")
train_dataset_raw = datasets.MNIST(root='./data', train=True, download=True)
print(train_dataset_raw.train_data.shape)
print(train_dataset_raw.train_data.float().mean(axis=(0,1,2))/255)
print(train_dataset_raw.train_data.float().std(axis=(0,1,2))/255)

print("\n FashionMNIST")
train_dataset_raw = datasets.FashionMNIST(root='./data', train=True, download=True)
print(train_dataset_raw.train_data.shape)
print(train_dataset_raw.train_data.float().mean(axis=(0,1,2))/255)
print(train_dataset_raw.train_data.float().std(axis=(0,1,2))/255)

MNIST
torch.Size([60000, 28, 28])
tensor(0.1307)
tensor(0.3081)

 FashionMNIST
torch.Size([60000, 28, 28])
tensor(0.2860)
tensor(0.3530)


In [23]:
print("\n CIFAR10")
train_dataset_raw = datasets.CIFAR10(root='./data', train=True, download=True)
print(train_dataset_raw.data.shape)
print(train_dataset_raw.data.mean(axis=(0,1,2))/255)
print(train_dataset_raw.data.std(axis=(0,1,2))/255)

print("\n CIFAR100")
train_dataset_raw = datasets.CIFAR100(root='./data', train=True, download=True)
print(train_dataset_raw.data.shape)
print(train_dataset_raw.data.mean(axis=(0,1,2))/255)
print(train_dataset_raw.data.std(axis=(0,1,2))/255)


 CIFAR10
Files already downloaded and verified
(50000, 32, 32, 3)
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]

 CIFAR100
Files already downloaded and verified
(50000, 32, 32, 3)
[0.50707516 0.48654887 0.44091784]
[0.26733429 0.25643846 0.27615047]
